In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,StandardScaler,Normalizer,OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score,precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
import seaborn as sns
from sklearn.metrics import roc_curve,auc
import itertools
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
data = pd.read_csv('preprocessed_data.csv')

In [50]:
data.columns

Index(['payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'customer_city', 'customer_state', 'price',
       'freight_value', 'product_name_length', 'product_description_length',
       'product_photos_qty', 'seller_state', 'product_category_name',
       'delivery_days', 'estimated_days', 'ships_in', 'arrival_time',
       'delivery_impression', 'estimated_del_impression', 'ship_impression',
       'seller_popularity', 'existing_cust', 'Score'],
      dtype='object')

In [49]:
print('Total unique category are ',data.seller_popularity.unique())

Total unique category are  [  58 1170  161  166  445  404    7 1388  232  350   20  567   12 1216
  679  345    5 1825  316    6   28   88   25   38  585   79  357  556
  391   15  303 1489   98  538   55  788   11  270   14  747   13  188
   84   16  243  154 1453  139   34  147  624  101  141    8   68  160
  189   40   27  177   86 2068   50 2088   23   62  125  263  214   71
   77  268   80   91  332  194   56   19  742  191  140   22   39  136
 1643  159  348 1174   37   47 1999   30  335    9  179   49   89  182
   93  138   18  190  208   33   48   99   61  131  561   74  144    4
   95  290  106    1   60  450   35  497   75  130   97  258  123  367
  135  129  210   10  794  100  863   78  428  169   72  346   17  400
  153 1218  337  217   69  108    3   96   92   87 1010   24  134  180
    2  540  264   94  424  221   21  597  203  358   64  133  360  117
   82  251  453  173  143   44  107  132  184  120   42  118  204  111
  216   63   65  186  193  333   57   46  412   66

In [3]:
#spliting data to train and test data
X = data.drop('Score',axis=1)
Y = data.Score.values

X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=.33,stratify=Y,random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(68542, 22) (33760, 22) (68542,) (33760,)


In [36]:
test = X_test.iloc[:10]

In [37]:
test

,payment_sequential,payment_type,payment_installments,payment_value,customer_city,customer_state,price,freight_value,product_name_length,product_description_length,...,product_category_name,delivery_days,estimated_days,ships_in,arrival_time,delivery_impression,estimated_del_impression,ship_impression,seller_popularity,existing_cust
71115,1,credit_card,10,143.10,sao paulo,SP,134.00,9.10,56.0,1498.0,...,auto,2,4,2,Early/OnTime,Very_Fast,Very_Fast,Very_Fast,82,0
6722,1,credit_card,8,102.13,sao jose,SC,79.00,23.13,59.0,206.0,...,stationery,5,23,4,Early/OnTime,Very_Fast,Neutral,Fast,1174,0
75363,1,credit_card,3,107.72,ibiuna,SP,89.00,18.72,35.0,920.0,...,fashion_bags_accessories,18,43,14,Early/OnTime,Neutral,Worst,Neutral,34,0
76672,1,credit_card,1,57.68,sao paulo,SP,49.90,7.78,57.0,804.0,...,auto,1,16,4,Early/OnTime,Very_Fast,Neutral,Fast,10,0
6941,1,credit_card,8,337.23,lauro de freitas,BA,299.99,37.24,43.0,1002.0,...,housewares,17,29,5,Early/OnTime,Neutral,Slow,Fast,47,0
89165,1,credit_card,4,195.69,petropolis,RJ,178.00,17.69,48.0,865.0,...,watches_gifts,18,26,6,Early/OnTime,Neutral,Slow,Fast,86,0
5011,1,credit_card,6,87.73,sao paulo,SP,79.43,8.30,56.0,901.0,...,computers_accessories,7,19,7,Early/OnTime,Very_Fast,Neutral,Fast,110,0
505,1,boleto,1,36.52,santa juliana,MG,22.00,14.52,60.0,704.0,...,computers_accessories,16,29,4,Early/OnTime,Neutral,Slow,Fast,412,0
78431,1,credit_card,5,229.68,otacilio costa,SC,95.20,19.64,57.0,819.0,...,construction_tools_construction,17,32,6,Early/OnTime,Neutral,Slow,Fast,22,1
66230,1,boleto,1,106.78,cariacica,ES,89.90,16.88,56.0,701.0,...,furniture_decor,14,27,8,Early/OnTime,Fast,Slow,Neutral,88,0


In [41]:
onehot = CountVectorizer()
#onehot = OneHotEncoder()
#payment_type
payment_type_train = onehot.fit(X_train.payment_type.values)
payment_type_test = onehot.transform(test.payment_type.values)

In [42]:
print(payment_type_test)

  (0, 1)	1
  (1, 1)	1
  (2, 1)	1
  (3, 1)	1
  (4, 1)	1
  (5, 1)	1
  (6, 1)	1
  (7, 0)	1
  (8, 1)	1
  (9, 0)	1


In [21]:
#initialising oneHotEncoder

onehot = CountVectorizer()
cat = OneHotEncoder()
#payment_type
payment_type_train = onehot.fit_transform(X_train.payment_type.values)
payment_type_test = onehot.transform(X_test.payment_type.values)


#customer_state
customer_state_train = onehot.fit_transform(X_train.customer_state.values)
customer_state_test = onehot.transform(X_test.customer_state.values)

#seller_state
seller_state_train = onehot.fit_transform(X_train.seller_state.values)
seller_state_test = onehot.transform(X_test.seller_state.values)


#product_category_name
product_category_name_train = onehot.fit_transform(X_train.product_category_name.values)
product_category_name_test = onehot.transform(X_test.product_category_name.values)


#arrival_time
arrival_time_train = onehot.fit_transform(X_train.arrival_time.values)
arrival_time_test = onehot.transform(X_test.arrival_time.values)

#delivery_impression
delivery_impression_train = onehot.fit_transform(X_train.delivery_impression.values)
delivery_impression_test = onehot.transform(X_test.delivery_impression.values)


#estimated_del_impression
estimated_del_impression_train = onehot.fit_transform(X_train.estimated_del_impression.values)
estimated_del_impression_test = onehot.transform(X_test.estimated_del_impression.values)

#ship_impression
ship_impression_train = onehot.fit_transform(X_train.ship_impression.values)
ship_impression_test = onehot.transform(X_test.ship_impression.values)


#existing_cust
existing_cust_train = cat.fit_transform(X_train.existing_cust.values.reshape(-1,1))
existing_cust_test = cat.transform(X_test.existing_cust.values.reshape(-1,1))

  (0, 1)	1
  (1, 1)	1
  (2, 1)	1
  (3, 1)	1
  (4, 1)	1
  (5, 1)	1
  (6, 1)	1
  (7, 0)	1
  (8, 1)	1
  (9, 0)	1
  (10, 1)	1
  (11, 1)	1
  (12, 1)	1
  (13, 1)	1
  (14, 1)	1
  (15, 1)	1
  (16, 1)	1
  (17, 1)	1
  (18, 1)	1
  (19, 0)	1


In [18]:
payment_type_test

<20x4 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [15]:
scaler = Normalizer()

#payment_sequential feature
payment_sequential_train = scaler.fit(X_train.payment_sequential.values.reshape(-1,1))


In [ ]:
scaler = Normalizer()
min_max = MinMaxScaler()

#payment_sequential feature
payment_sequential_train = scaler.fit(X_train.payment_sequential.values.reshape(-1,1))
payment_sequential_test = scaler.transform(test.payment_sequential.values.reshape(-1,1))

#payment_installments feature
payment_installments_train = scaler.fit(X_train.payment_installments.values.reshape(-1,1))
payment_installments_test = scaler.transform(test.payment_installments.values.reshape(-1,1))

#Payment value feature
payment_value_train = scaler.fit(X_train.payment_value.values.reshape(-1,1))
payment_value_test = scaler.transform(test.payment_value.values.reshape(-1,1))


#price
price_train = scaler.fit(X_train.price.values.reshape(-1,1))
price_test = scaler.transform(test.price.values.reshape(-1,1))

#freight_value
freight_value_train = scaler.fit(X_train.freight_value.values.reshape(-1,1))
freight_value_test = scaler.transform(test.freight_value.values.reshape(-1,1))

#product_name_length
product_name_length_train = scaler.fit(X_train.product_name_length.values.reshape(-1,1))
product_name_length_test = scaler.transform(test.product_name_length.values.reshape(-1,1))


#product_description_length
product_description_length_train = scaler.fit(X_train.product_description_length.values.reshape(-1,1))
product_description_length_test = scaler.transform(test.product_description_length.values.reshape(-1,1))


#product_photos_qty
product_photos_qty_train = scaler.fit(X_train.product_photos_qty.values.reshape(-1,1))
product_photos_qty_test = scaler.transform(test.product_photos_qty.values.reshape(-1,1))


#delivery_days
delivery_days_train = scaler.fit(X_train.delivery_days.values.reshape(-1,1))
delivery_days_test = scaler.transform(test.delivery_days.values.reshape(-1,1))


#estimated_days
estimated_days_train = scaler.fit(X_train.estimated_days.values.reshape(-1,1))
estimated_days_test = scaler.transform(test.estimated_days.values.reshape(-1,1))


#ships_in
ships_in_train = scaler.fit(X_train.ships_in.values.reshape(-1,1))
ships_in_test = scaler.transform(test.ships_in.values.reshape(-1,1))

#seller_popularity
seller_popularity_train = MinMaxScaler().fit(X_train.seller_popularity.values.reshape(-1,1))
seller_popularity_test = MinMaxScaler().transform(test.seller_popularity.values.reshape(-1,1))